In [2]:
import pandas as pd
file = "C:\HCL\iLA\Versions\Cisco_PSI_Versions\Backup\RawTraindata.xlsx"
df = pd.read_excel(file)
text_data = df["Content"].head(25).to_list()
# Category = df["ErrorCategory"].head(10).to_list()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Original data
# text_list = ['> Connect function on console failed: Unable to find prompt:',
#              '> /auto/cafy/release/23.03.13/rhel7-23.03.13/lib/python3.6/site-packages/netmiko/base_connection.py:978: ValueError: Unable to find prompt:',
#              '> /usr/cisco/packages/python/python-3.6.7/lib/python3.6/socket.py:713: ConnectionRefusedError: [Errno 111] Connection refused',
#              '> Invalid show processes output:execute failed on vty as session disconnected or did not connect successfully',
#              '> execute failed on vty as session disconnected or did not connect successfully',
#              '> f3b-r1-pod9: Error detecting calvados nodes: execute failed on vty as session disconnected or did not connect successfully',
#              "> Commit failed with the following errors:  commit  Mon Mar 13 00:50:17.473 UTC  % Failed to commit one or more configuration ",
#              '> verify_multicast_traffic took longer than expected. Timeout:(210)s Elapsed(352)s',
#              '> Convergence not met criteria']

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_list)
total_words = len(tokenizer.word_index) + 1

# Create input sequences using tokenized text
input_sequences = []
for line in text_list:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences for uniform input size
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
X, y = input_sequences[:,:-1],input_sequences[:,-1]

# Convert labels to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Define the model architecture
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=100, verbose=1)

# Generate synthetic data
# Generate synthetic data
# Generate synthetic data
# Generate synthetic data
# Generate synthetic data
def generate_text(seed_text, next_words, max_sequence_len, model):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list)[0]
        
        # Apply nucleus sampling
        sorted_indices = np.argsort(predicted_probs)[::-1]
        cumulative_probs = np.cumsum(predicted_probs[sorted_indices])
        threshold_index = np.where(cumulative_probs > 0.95)[0][0]
        sampled_index = np.random.choice(sorted_indices[:threshold_index])
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == sampled_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

seed_text = "Connect function on console failed:"
generated_text = generate_text(seed_text, 50, max_sequence_len, model)
print(generated_text)

Epoch 1/100
42/42 [==============================] - 297s 7s/step - loss: 5.8940 - accuracy: 0.0219
Epoch 2/100
42/42 [==============================] - 298s 7s/step - loss: 5.5092 - accuracy: 0.0203
Epoch 3/100
42/42 [==============================] - 301s 7s/step - loss: 5.4556 - accuracy: 0.0226
Epoch 4/100
42/42 [==============================] - 286s 7s/step - loss: 5.3049 - accuracy: 0.0301
Epoch 5/100
42/42 [==============================] - 288s 7s/step - loss: 5.0942 - accuracy: 0.0399
Epoch 6/100
42/42 [==============================] - 272s 6s/step - loss: 4.8533 - accuracy: 0.0603
Epoch 7/100
42/42 [==============================] - 276s 7s/step - loss: 4.6246 - accuracy: 0.0882
Epoch 8/100
42/42 [==============================] - 310s 7s/step - loss: 4.4282 - accuracy: 0.1093
Epoch 9/100
42/42 [==============================] - 316s 8s/step - loss: 4.2308 - accuracy: 0.1281
Epoch 10/100
42/42 [==============================] - 307s 7s/step - loss: 4.0619 - accuracy: 0.1341

Epoch 82/100
42/42 [==============================] - 372s 9s/step - loss: 0.2508 - accuracy: 0.9729
Epoch 83/100
42/42 [==============================] - 379s 9s/step - loss: 0.2397 - accuracy: 0.9721
Epoch 84/100
42/42 [==============================] - 383s 9s/step - loss: 0.2299 - accuracy: 0.9721
Epoch 85/100
42/42 [==============================] - 391s 9s/step - loss: 0.2231 - accuracy: 0.9736
Epoch 86/100
42/42 [==============================] - 370s 9s/step - loss: 0.2158 - accuracy: 0.9751
Epoch 87/100
21/42 [==============>...............] - ETA: 3:13 - loss: 0.2122 - accuracy: 0.9717

In [3]:
# !pip install gpt-2-simple
import gpt_2_simple as gpt2


# text_data = ['> Connect function on console failed: Unable to find prompt:',
#  '> /auto/cafy/release/23.03.13/rhel7-23.03.13/lib/python3.6/site-packages/netmiko/base_connection.py:978: ValueError: Unable to find prompt:',
#  '> /usr/cisco/packages/python/python-3.6.7/lib/python3.6/socket.py:713: ConnectionRefusedError: [Errno 111] Connection refused',
#  '> Invalid show processes output:execute failed on vty as session disconnected or did not connect successfully',
#  '> execute failed on vty as session disconnected or did not connect successfully',
#  '> f3b-r1-pod9: Error detecting calvados nodes: execute failed on vty as session disconnected or did not connect successfully',
#  "> Commit failed with the following errors:  commit  Mon Mar 13 00:50:17.473 UTC  % Failed to commit one or more configuration ",
#  '> verify_multicast_traffic took longer than expected. Timeout:(210)s Elapsed(352)s',
#  '> Convergence not met criteria']

# Combine the list of texts into a single string
input_text = "\n".join(text_data)

# Download and load the GPT-2 model
model_name = "124M"  # You can use "355M" or "774M" for larger models
gpt2.download_gpt2(model_name=model_name)

# Start a TensorFlow session
sess = gpt2.start_tf_sess()

# Fine-tune the GPT-2 model with your input text
gpt2.finetune(sess, input_text, model_name=model_name, steps=1000)

# Generate synthetic text
generated_text = gpt2.generate(sess, model_name=model_name, return_as_list=True)[0]

# Print the generated text
print("Generated Text:\n", generated_text)


Fetching checkpoint: 1.05Mit [00:00, ?it/s]                                                         
Fetching encoder.json: 1.05Mit [00:01, 532kit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 1.91Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [06:16, 1.32Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, ?it/s]                                                   
Fetching model.ckpt.meta: 1.05Mit [00:01, 892kit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:01, 604kit/s]                                                       


Loading checkpoint models\124M\model.ckpt
INFO:tensorflow:Restoring parameters from models\124M\model.ckpt
Loading dataset...


0it [00:00, ?it/s]

dataset has 0 tokens
Training...


ZeroDivisionError: integer division or modulo by zero

In [8]:
import torch
from transformers import pipeline

# Load the pre-trained model for question answering
nlp_model = "bert-large-uncased-whole-word-masking-finetuned-squad"
qa_pipeline = pipeline("question-answering", model=nlp_model, tokenizer=nlp_model)

def extract_information_from_text(text, question):
    """
    Function to extract information from the given text using a specified question.
    
    Args:
    - text: The input text from which information is to be extracted.
    - question: The prompt/question used for information extraction.
    
    Returns:
    - extracted_information: Extracted information as a dictionary containing 'answer' and 'score'.
    """
    # Tokenize the text and question
    inputs = qa_pipeline.tokenizer(text, question, return_tensors="pt", truncation=True, padding=True)

    # Perform question answering
    outputs = qa_pipeline.model(**inputs)
    
    # Decode the predicted answer
    start_idx = torch.argmax(outputs.start_logits)
    end_idx = torch.argmax(outputs.end_logits) + 1
    answer = qa_pipeline.tokenizer.convert_tokens_to_string(qa_pipeline.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx]))
    
    # Calculate confidence score
    score = torch.max(outputs.start_logits) + torch.max(outputs.end_logits)
    
    extracted_information = {
        "answer": answer,
        "score": score.item()
    }
    
    return extracted_information

# Example usage
input_text = "0a LINUX-0-uoamapp <2020-09-20T10:45:16.471784Z> 139F ERR/LTX/MED_Generic, PowerSupplyManager::areVoltageAlarmsActive: alarms related with voltage are inactive"
prompt_question = "Generte similar error message?"

extracted_info = extract_information_from_text(input_text, prompt_question)
print("Extracted information:", extracted_info)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Extracted information: {'answer': 'linux - 0 - uoamapp', 'score': -5.3858747482299805}


In [4]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.today()
print(type(current_datetime))
# Format the date and time as desired
formatted_datetime = current_datetime.strftime('%Y-%m-%d %H:%M:%S')

# Print the formatted date and time
print(formatted_datetime)


<class 'datetime.datetime'>
2024-04-08 17:34:09


In [5]:
from datetime import datetime

formatted_datetime = datetime.today().strftime('%Y-%m-%d %H:%M:%S')

print(formatted_datetime)


2024-04-08 17:34:35


In [ ]:
def trainErrorClassModel(ToolPath):
    df = oslib.readDataFiles(ToolPath+trainDataFile)
    df['Regex_Cleaned'] = df['Content'].map(oslib.remove_dicts_lists)
    df['Regex_Cleaned'] = df['Regex_Cleaned'].map(oslib.cleanData)
    df['Regex_Cleaned'] = df['Regex_Cleaned'].apply(lambda x: oslib.remove_day_month(x))
    # Apply the function to the specified column
    
    cleaned_list = df['Regex_Cleaned'] 
    ignore_stop_words = ["or", "and"]
    # cleaned_stop_data = oslib.RemoveStopWords(cleaned_list, ignore_stop_words)
    # df["StopWords_cleaned_data"] = cleaned_stop_data
    df['MaskedContent'] = df['Regex_Cleaned'].apply(lambda x: oslib.strip_and_join(x))
    df.to_excel(ToolPath+trainDataFile,index=False)
    print("Masked the Content completed")

In [ ]:
train_file = f"LogA/Data/oneshot/RawTraindata.xlsx"
        df = oneoslib.readDataFiles(train_file)
        # cleaned_string = remove_dicts_lists(input_string)
        df['Regex_Cleaned'] = df['Content'].map(oneoslib.remove_dicts_lists)
        df['Regex_Cleaned'] = df['Content'].map(oneoslib.cleanData)
        df['Regex_Cleaned'] = df['Regex_Cleaned'].apply(lambda x: oneoslib.remove_day_month(x))
        # Apply the function to the specified column
        
        cleaned_list = df['Regex_Cleaned'] 
        ignore_stop_words = ["or", "and"]
        # cleaned_stop_data = oneoslib.RemoveStopWords(cleaned_list, ignore_stop_words)
        # df["StopWords_cleaned_data"] = cleaned_stop_data
        df['MaskedContent'] = df['Regex_Cleaned'].apply(lambda x: oneoslib.strip_and_join(x))
        df.to_excel(train_file,index=False)

In [ ]:

        # test_set_error['MaskedContent'] = test_set_error['Content'].map(oneoslib.cleanData)
        test_set_error['Regex_Cleaned'] = test_set_error['Content'].map(oneoslib.remove_dicts_lists)
        test_set_error['Regex_Cleaned'] = test_set_error['Regex_Cleaned'].map(oneoslib.cleanData)
        test_set_error['Regex_Cleaned'] = test_set_error['Regex_Cleaned'].apply(lambda x: oneoslib.remove_day_month(x))
        # Apply the function to the specified column
        
        cleaned_list = test_set_error['Regex_Cleaned'] 
        ignore_stop_words = ["or", "and"]
        # cleaned_stop_data = oneoslib.RemoveStopWords(cleaned_list, ignore_stop_words)
        # test_set_error["StopWords_cleaned_data"] = cleaned_stop_data
        test_set_error['MaskedContent'] = test_set_error['Regex_Cleaned'].apply(lambda x: oneoslib.strip_and_join(x))
        